In [18]:
import pandas as pd
from Poloniex import Poloniex as p
from Bitfinex import Bitfinex as b
from Bittrex import Bittrex as bt
from Cryptopia import Cryptopia as c
import multiprocessing


In [19]:
def flatten(nested_list):
    return [item for sublist in nested_list for item in sublist]

def get_symbols():
    plist = [key.split("_") for key in p.get_ticker().keys()]
    blist = [[symbol[:3].upper(), symbol[3:].upper()] for symbol in b.get_symbols()]

    pset = set(flatten(plist))
    bset = set(flatten(blist))
    common = pset & bset

    plist2 = filter(lambda a: len(common & set(a))==2, [i[::k] for i in plist for k in [1, -1]])
    blist2 = filter(lambda a: len(common & set(a))==2, [i[::k] for i in blist for k in [1, -1]])

    plist3 = filter(lambda a: a[0] == u"BTC", plist2)
    blist3 = filter(lambda a: a[0] == u"BTC", blist2)

    exchangeList = plist3
    symbols = [a[1] for a in exchangeList]

    return symbols

In [47]:
def fold_depth(orders):
    def fold2(order1, order2):
        return [(order1[0] * order1[1] + order2[0] * order2[1]) / (order1[1] + order2[1]),
               (order1[1] + order2[1])]
    
    def temp(i=0, result=[]):
        if i == len(btest2):
            return result
        elif i == 0:
            result.append(fold2( btest2[i], [0, 0]))
            return temp(i+1, result)
        else:
            result.append(fold2(btest2[i], result[-1]))
            return temp(i+1, result)
    return temp()


def get_orderbook(symbol, ex="all", depth=1, depthFold=False):
    def get_orderbook_p(symbol, depth, depthFold=False):
        pOrders = p.get_orderbook("BTC_" + symbol.upper(), depth)
        if depthFold:
            return {"asks": fold_depth([[float(i[0]), float(i[1])] for i in pOrders["asks"]]), 
                    "bids": fold_depth([[float(i[0]), float(i[1])] for i in pOrders["bids"]])}
        return {"asks": [[float(i[0]), float(i[1])] for i in pOrders.get("asks")], 
                "bids": [[float(i[0]), float(i[1])] for i in pOrders.get("bids")]}

    def get_orderbook_b(symbol, depth, depthFold=False):
        bOrders = b.get_orderbook(symbol.lower() + "btc", depth)
        if depthFold:
            return {"asks": fold_depth([[float(i["price"]), float(i["amount"])] for i in bOrders["asks"]]), 
                    "bids": fold_depth([[float(i["price"]), float(i["amount"])] for i in bOrders["bids"]])}
        return {"asks": [[float(i["price"]), float(i["amount"])] for i in bOrders.get("asks")], 
                "bids": [[float(i["price"]), float(i["amount"])] for i in bOrders.get("bids")]}
    
    def get_orderbook_bt(symbol, depth, depthFold=False):
        symbol = symbol.upper() if symbol.upper() != "BCH" else "BCC"
        btOrders = bt.get_orderbook("BTC-" + symbol)
                
        asks = [[float(item["Rate"]), float(item["Quantity"])] for item in btOrders["sell"][:depth]]
        bids = [[float(item["Rate"]), float(item["Quantity"])] for item in btOrders["buy"][:depth]]
                
        if depthFold:
            return {"asks": fold_depth(asks),
                    "bids": fold_depth(bids)}
        return {"asks": asks, 
                "bids": bids}
    
    def get_orderbook_with_command(command, symbol, depth, depthFold=False):
        if command == "p":
            return get_orderbook_p(symbol, depth, depthFold)
        elif command == "b":
            return get_orderbook_b(symbol, depth, depthFold)
        else:
            return get_orderbook_bt(symbol, depth, depthFold)
        
    if ex == "all":
        result = map(lambda a: [a, get_orderbook_with_command(a, symbol, depth, depthFold)], 
                          exs)
        return dict(result)
    elif ex == "p":
        return {"p": get_orderbook_p(symbol, depth, depthFold)}
    elif ex == "b":
        return {"b": get_orderbook_b(symbol, depth, depthFold)}
    else:
        return {"bt": get_orderbook_bt(symbol, depth, depthFold)}
        

def get_depth1_opportunity(symbols, exs=exs):
    temp = {symbol: get_orderbook(symbol, depth=1) for symbol in symbols}
    exs = exs
    
    index = [item for item in temp] # 货币标记
    
#     results = {ex + "_" + type: [temp[item][ex][type][0][0] for item in index] 
#                    for ex in exs
#                       for type in ["asks", "bids"]}
    results = {}
    for t in ["asks", "bids"]:
        for ex in exs:
                results[ex + "_" + t] = [temp[currency][ex][t][0][0] for currency in index]
    
    resultsDF = pd.DataFrame(results)
    resultsDF.index = index
    
    for sell_to in exs:
        for buy_from in exs:
            if sell_to == buy_from:
                continue
            else:
                resultsDF[buy_from.upper() + "_ask" + "=>" + sell_to.upper() + "_bid" ] = (resultsDF[sell_to + "_bids"] - resultsDF[buy_from + "_asks"]) / resultsDF[sell_to + "_bids"] * 100
    
    return resultsDF

In [43]:
get_orderbook("BCH", ex="all")

{'bt': {'asks': [[0.09351799, 10.546]], 'bids': [[0.09331812, 2.12415959]]},
 'p': {'asks': [[0.09308929, 1.2057044]], 'bids': [[0.09280676, 29.64]]}}

In [49]:
get_depth1_opportunity(["BCH", "LTC", "ETC", "ETH", "ZEC", "XMR", "XRP", "OMG"])

,bt_asks,bt_bids,p_asks,p_bids,BT_ask=>P_bid,P_ask=>BT_bid
ZEC,0.057700,0.057698,0.057649,0.057412,-0.500797,0.084873
BCH,0.093147,0.093025,0.093346,0.093095,-0.056340,-0.344553
LTC,0.012245,0.012226,0.012250,0.012242,-0.026384,-0.196139
ETC,0.002873,0.002861,0.002869,0.002866,-0.249468,-0.292208
OMG,0.002170,0.002165,0.002174,0.002174,0.197765,-0.427704
ETH,0.067850,0.067705,0.067634,0.067623,-0.335403,0.105369
XRP,0.000046,0.000046,0.000046,0.000046,-0.174558,-0.108932
XMR,0.020800,0.020700,0.020710,0.020700,-0.482947,-0.047971
